In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [73]:
import numpy as np
import pandas as pd
import nbgwas
from nbgwas.nbgwas import Nbgwas

In [32]:
nbgwas.__version__

'0.1.0'

In [17]:
network = Nbgwas._load_pcnet()

In [20]:
g = Nbgwas(
        gene_level_summary='../data/interim/scz/gene_level_summary_stats_pmid_25056061.txt',
        network=network,
    )

In [22]:
g.gene_level_summary.head()

,Gene,Chr,Gene Start,Gene End,nSNPs,TopSNP,TopSNP Pos,TopSNP P-Value,SNP Distance
0,HIST1H4K,6,27906930,27907284,8,rs34706883,27913234.0,5.071180e-10,6304.0
1,HIST1H2AK,6,27913636,27914096,16,rs34706883,27913234.0,5.071180e-10,402.0
2,HIST1H2BN,6,27914418,27914867,17,rs34706883,27913234.0,5.071180e-10,1184.0
3,HIST1H2AL,6,27941085,27941555,10,rs13199772,27942064.0,7.053790e-10,979.0
4,HIST1H1B,6,27942548,27943338,10,rs13199772,27942064.0,7.053790e-10,484.0


In [26]:
g.diffuse(
    kernel='/cellar/users/decarlin/Data/kernels/PCNet_kernel.hdf'
)

/cellar/users/samsonfong/bin/anaconda3/envs/nbgwas/lib/python3.6/site-packages/pandas-0.23.4-py3.6-linux-x86_64.egg/pandas/core/indexing.py:1027: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


In [27]:
g.boosted_pvalues.head()

,prop
Gene,
TCF4,0.479835
HIST1H4C,0.467925
HIST1H2BC,0.467749
HIST1H2BN,0.466517
HIST1H4J,0.466316


In [28]:
g.heat_diffusion()

In [29]:
g.boosted_pvalues.head()

,prop
Gene,
LY6G6F,0.744055
MUSTN1,0.742368
AGBL3,0.616114
PATE1,0.501521
ZSCAN23,0.080481


In [38]:
df = g.gene_level_summary.copy()

In [116]:
pvals = df.set_index('Gene')['TopSNP P-Value']
#genes, pvals = np.array(pvals.index), pvals.values

In [121]:
pvals.reindex(g.kernel)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [60]:
pvals[pvals > 5e-6] = 0
pvals[pvals < 5e-6] = 1

In [63]:
np.dot(pvals, g.kernel.loc[genes, genes])

/cellar/users/samsonfong/bin/anaconda3/envs/nbgwas/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


array([nan, nan, nan, ..., nan, nan, nan])

In [64]:
g.laplacian

<19781x19781 sparse matrix of type '<class 'numpy.int64'>'
	with 5469229 stored elements in Compressed Sparse Column format>

In [89]:
name='prop'
threshold=5e-6
threshold_genes = {}
prop_vectors = []

network_genes = list(g.kernel.index)

In [90]:
threshold_genes[name] = g.gene_level_summary[g.gene_level_summary['TopSNP P-Value'] < threshold]
prop_vector = (g.gene_level_summary.set_index('Gene').loc[network_genes, 'TopSNP P-Value'] < threshold).astype(float)
prop_vector.name = name
prop_vectors.append(prop_vector)
prop_vector_matrix = pd.concat(prop_vectors, axis=1).loc[network_genes].T

/cellar/users/samsonfong/bin/anaconda3/envs/nbgwas/lib/python3.6/site-packages/pandas-0.23.4-py3.6-linux-x86_64.egg/pandas/core/indexing.py:1027: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


In [91]:
threshold_genes['prop'].shape

(66, 9)

In [92]:
g.gene_level_summary.shape

(14966, 9)

In [96]:
prop_vector_matrix.sum(axis=1)

prop    66.0
dtype: float64

In [97]:
prop_vector_matrix.shape

(1, 19781)

In [ ]:
#propagate with pre-computed kernel
prop_val_matrix = np.dot(prop_vector_matrix, self.kernel)
prop_val_table = pd.DataFrame(prop_val_matrix, index = prop_vector_matrix.index, columns = prop_vector_matrix.columns)

In [115]:
prop_vector_matrix.isnull().any(axis=1)

prop    False
dtype: bool

In [108]:
binarized_pvals = (pvals < 5e-6).astype(float)[np.newaxis, ...]

In [111]:
np.dot(binarized_pvals)

ValueError: shapes (1,14966) and (19781,19781) not aligned: 14966 (dim 1) != 19781 (dim 0)